In [1]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import re

# Berlin

In [2]:
# Get Berlin Metro size
url = "https://en.wikipedia.org/wiki/Berlin#cite_note-145"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
res = soup.find("div", class_="mw-parser-output").findAll("table", class_="wikitable")[4]

In [3]:
res = soup.find("div", class_="mw-parser-output").findAll("table", class_="wikitable")[4]#.findAll("td")

In [4]:
# Re text and values
text = str(res)
nr_pattern = "(\d?,?\d{1,})\\xa0km"
txt_pattern = "\>(\w+(\s|\W)?\w+)\</a>"
ber_bahn_val = re.findall(nr_pattern, text)
ber_bahn_txt = re.findall(txt_pattern, text)
a,b = map(list, zip(*ber_bahn_txt))
val_tram = ["S-Bahn", "U-Bahn", "Tram", "Bus"]
val_res = []
for i in a:
    if i in val_tram:
        val_res.append(i)

In [5]:
ber_bahn_val[0] = float(str(ber_bahn_val[0]).replace(",",""))
ber_bahn_val[1] = float(str(ber_bahn_val[1]))
ber_bahn_val[2] = float(str(ber_bahn_val[2]))
ber_bahn_val[3] = float(str(ber_bahn_val[3]).replace(",",""))

In [6]:
berl_values = set(zip(val_res, list(ber_bahn_val)))
berl_values

{('Bus', 1675.0), ('S-Bahn', 331.0), ('Tram', 194.0), ('U-Bahn', 146.0)}

# Madrid

In [23]:
# Bus
url = "https://www.crtm.es/tu-transporte-publico/autobuses-emt"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
result = soup.findAll("div", class_="detalle1")

In [29]:
text_mad = str(result)
mad_pattern = "\s+(\d+.\d+)\s+"
mad_val_bus = re.findall(mad_pattern, text_mad)[1]
mad_val_bus=float(mad_val_bus.replace('.',''))
mad_val_bus 

3562.0

In [9]:
# S-Bahn
url = "https://www.crtm.es/tu-transporte-publico/metro-ligero.aspx"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
result = soup.findAll("div", class_="detalle1")

In [10]:
text_mad = str(result)
mad_pattern = "\s(\d+.\d+)\s"
mad_val_s = re.findall(mad_pattern, text_mad)[0]
mad_val_s

'35,4'

In [11]:
# Metro
url_3 = "https://en.wikipedia.org/wiki/Madrid"
response_3 = requests.get(url_3)
soup_3 = BeautifulSoup(response_3.content, "html.parser")
result_3 = soup_3.find("div", class_="mw-parser-output").findAll("p") #cite_ref-transport_170-0

In [12]:
text_mad_3 = str(result_3)
mad_pattern = "at\s(\d+.\d+)\s" #"(\w)\s\k\i"
mad_val_metro = re.findall(mad_pattern, text_mad_3)[2]
mad_val_metro = float(mad_val_metro)
mad_val_metro = float(mad_val_metro)
mad_val_metro

294.0

In [13]:
# metro cercanieas
url_4 = "https://en.wikipedia.org/wiki/Madrid"
response_4 = requests.get(url_4)
soup_4 = BeautifulSoup(response_4.content, "html.parser")
result_4 = soup_4.find("div", class_="mw-parser-output").findAll("p") #cite_ref-transport_170-0

In [14]:
text_mad = str(result_4)
mad_pattern = "\s+(\d+.\d+)\s"
mad_val_cercanias = re.findall(mad_pattern, text_mad)[0]
mad_val_cercanias = float(mad_val_cercanias.replace(",",""))

In [15]:
# cercanias + S-Bahn 
mad_val_s2=mad_val_s
mad_val_sf=float(mad_val_s2.replace(",",""))
mad__val_SC = mad_val_sf + mad_val_cercanias
mad__val_SC

357.3

# Hong Kong

In [16]:
df_hk = pd.DataFrame([[70.4, "sum"],[1685.6, "Bus"]], columns=["hk_km", "transport"])
df_hk = df_hk.sum()[0]
df_hk

1756.0

# Sum df

In [30]:
df_mad = pd.DataFrame([mad_val_bus, mad__val_SC, mad_val_metro], columns=["mad_km"])
df_mad["transport"] = ["Bus", "S-Bahn", "Metro"]
df_mad = df_mad.sum()[0]
df_mad

4213.3

In [18]:
df_ber = pd.DataFrame(berl_values, columns=["transport", "ber_km"])
df_ber = df_ber.sum()[1]
df_ber

2346.0

In [31]:
trans_km = {'city':['hong kong','madrid','berlin'],
             'len_pt':[df_hk,df_mad,df_ber]}
trans_km = pd.DataFrame(trans_km)
trans_km


,city,len_pt
0,hong kong,1756.0
1,madrid,4213.3
2,berlin,2346.0


In [32]:
trans_km.to_pickle("./pickles/transport_km.pkl")